In [1]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from textblob import TextBlob
import pandas as pd
import string
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import SnowballStemmer
from textblob import TextBlob
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\punch\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\punch\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\punch\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\punch\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\punch\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**Importing Dataset**

In [2]:
full_df = pd.read_csv("dataset.csv")
full_df

,scoreTeacher,studentId,question,keyAnswer,studentAnswer
0,3.5,[6],What is the role of a prototype program in pro...,To sumulate the behaviour of portions of the d...,High risk problems are address in the prototyp...
1,5.0,[5],What is the role of a prototype program in pro...,To simulate the behaviour of portions of the d...,To sumulate portions of the desired final prod...
2,4.0,[8],What is the role of a prototype program in pro...,To simulate the behaviour of portions of the d...,A prototype program simulates the behaviors of...
3,5.0,[3],What is the role of a prototype program in pro...,To simulate the behaviour of portions of the d...,Defined in the Specification phase a prototype...
4,3.0,[4],What is the role of a prototype program in pro...,To simulate the behaviour of portions of the d...,It's used to let the users have a first idea o...
...,...,...,...,...,...
408,3.5,[26],What is the role of a header-file?,"To store a class interface, including data mem...",Allow compiler to recognize the classes when u...
409,3.0,[1],What is the role of a header-file?,"To store a class interface, including data mem...",To promote function reusuablitly
410,3.0,[28],What is the role of a header-file?,"To store a class interface, including data mem...",class definitions are placed here
411,4.5,[15],What is the role of a header-file?,"To store a class interface, including data mem...",A header file usually contains class and/or fu...


**Text Preprocessing**

Spelling Correction

In [3]:
# Spelling Correction

full_df["keyAnswer"] = full_df["keyAnswer"].astype(str)
full_df["keySpell"] = full_df["keyAnswer"].apply(lambda txt: ''.join(TextBlob(txt).correct()))

In [4]:
full_df["studentAnswer"] = full_df["studentAnswer"].astype(str)
full_df["studentSpell"] = full_df["studentAnswer"].apply(lambda txt: ''.join(TextBlob(txt).correct()))

In [5]:
full_df[['keyAnswer','keySpell']]

,keyAnswer,keySpell
0,To sumulate the behaviour of portions of the d...,To simulate the behaviour of portions of the d...
1,To simulate the behaviour of portions of the d...,To simulate the behaviour of portions of the d...
2,To simulate the behaviour of portions of the d...,To simulate the behaviour of portions of the d...
3,To simulate the behaviour of portions of the d...,To simulate the behaviour of portions of the d...
4,To simulate the behaviour of portions of the d...,To simulate the behaviour of portions of the d...
...,...,...
408,"To store a class interface, including data mem...","To store a class interface, including data mem..."
409,"To store a class interface, including data mem...","To store a class interface, including data mem..."
410,"To store a class interface, including data mem...","To store a class interface, including data mem..."
411,"To store a class interface, including data mem...","To store a class interface, including data mem..."


Converting Abbreviation

In [6]:
contractions_dict = {
    "'s":" is",
    "ain't": "is not",
    "aren't": "are not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "I'd": "I would",
    "I'd've": "I would have",
    "I'll": "I will",
    "I'll've": "I will have",
    "I'm": "I am",
    "I've": "I have",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have", 
    }

In [7]:
# Converting Abbreviation

contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))
def expand_contractions(text,contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, text)

full_df['studentSpell']=full_df['studentSpell'].apply(lambda x:expand_contractions(x))
full_df['keySpell']=full_df['keySpell'].apply(lambda x:expand_contractions(x))

In [8]:
full_df[['studentAnswer','studentSpell']]

,studentAnswer,studentSpell
0,High risk problems are address in the prototyp...,High risk problems are address in the prototyp...
1,To sumulate portions of the desired final prod...,To simulate portions of the desired final prod...
2,A prototype program simulates the behaviors of...,A prototype program simulates the behavior of ...
3,Defined in the Specification phase a prototype...,Defined in the Specification phase a prototype...
4,It's used to let the users have a first idea o...,It is used to let the users have a first idea ...
...,...,...
408,Allow compiler to recognize the classes when u...,Allow compilers to recognize the classes when ...
409,To promote function reusuablitly,To promote function reusuablitly
410,class definitions are placed here,class definitions are placed here
411,A header file usually contains class and/or fu...,A header file usually contains class and/or fu...


Case Folding

In [9]:
#Case Folding

full_df["keyLower"] = full_df["keySpell"].str.lower()
full_df["studentLower"] = full_df["studentSpell"].str.lower()

full_df[['studentSpell', 'studentLower']]

,studentSpell,studentLower
0,High risk problems are address in the prototyp...,high risk problems are address in the prototyp...
1,To simulate portions of the desired final prod...,to simulate portions of the desired final prod...
2,A prototype program simulates the behavior of ...,a prototype program simulates the behavior of ...
3,Defined in the Specification phase a prototype...,defined in the specification phase a prototype...
4,It is used to let the users have a first idea ...,it is used to let the users have a first idea ...
...,...,...
408,Allow compilers to recognize the classes when ...,allow compilers to recognize the classes when ...
409,To promote function reusuablitly,to promote function reusuablitly
410,class definitions are placed here,class definitions are placed here
411,A header file usually contains class and/or fu...,a header file usually contains class and/or fu...


Removing Punctuation

In [10]:
#Removing Punctuation

PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

full_df["keyPunct"] = full_df["keyLower"].apply(lambda text: remove_punctuation(text))
full_df["studentPunct"] = full_df["studentLower"].apply(lambda text: remove_punctuation(text))

full_df[['studentLower', 'studentPunct']]

,studentLower,studentPunct
0,high risk problems are address in the prototyp...,high risk problems are address in the prototyp...
1,to simulate portions of the desired final prod...,to simulate portions of the desired final prod...
2,a prototype program simulates the behavior of ...,a prototype program simulates the behavior of ...
3,defined in the specification phase a prototype...,defined in the specification phase a prototype...
4,it is used to let the users have a first idea ...,it is used to let the users have a first idea ...
...,...,...
408,allow compilers to recognize the classes when ...,allow compilers to recognize the classes when ...
409,to promote function reusuablitly,to promote function reusuablitly
410,class definitions are placed here,class definitions are placed here
411,a header file usually contains class and/or fu...,a header file usually contains class andor fun...


Stopword Removal

In [11]:
#Stopword Removal

STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

full_df["keyStopword"] = full_df["keyPunct"].apply(lambda text: remove_stopwords(text))
full_df["studentStopword"] = full_df["studentPunct"].apply(lambda text: remove_stopwords(text))

full_df[['studentPunct', 'studentStopword']]

,studentPunct,studentStopword
0,high risk problems are address in the prototyp...,high risk problems address prototype program m...
1,to simulate portions of the desired final prod...,simulate portions desired final product quick ...
2,a prototype program simulates the behavior of ...,prototype program simulates behavior portions ...
3,defined in the specification phase a prototype...,defined specification phase prototype stimulat...
4,it is used to let the users have a first idea ...,used let users first idea completed program al...
...,...,...
408,allow compilers to recognize the classes when ...,allow compilers recognize classes used elsewhere
409,to promote function reusuablitly,promote function reusuablitly
410,class definitions are placed here,class definitions placed
411,a header file usually contains class andor fun...,header file usually contains class andor funct...


Stemming

In [12]:
#Stemming

stemmer = SnowballStemmer("english")  
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

full_df["studentStem"] = full_df["studentStopword"].apply(lambda text: stem_words(text))
full_df["keyStem"] = full_df["keyStopword"].apply(lambda text: stem_words(text))

full_df[['studentStopword','studentStem']]

,studentStopword,studentStem
0,high risk problems address prototype program m...,high risk problem address prototyp program mak...
1,simulate portions desired final product quick ...,simul portion desir final product quick easi p...
2,prototype program simulates behavior portions ...,prototyp program simul behavior portion desir ...
3,defined specification phase prototype stimulat...,defin specif phase prototyp stimul behavior po...
4,used let users first idea completed program al...,use let user first idea complet program allow ...
...,...,...
408,allow compilers recognize classes used elsewhere,allow compil recogn class use elsewher
409,promote function reusuablitly,promot function reusuablit
410,class definitions placed,class definit place
411,header file usually contains class andor funct...,header file usual contain class andor function...


POS Tagging

In [13]:
studentSplit = full_df["studentStopword"]
keySplit = full_df["keyStopword"]

studentSplitWords = [ nltk.word_tokenize( str(sentence) ) for sentence in studentSplit ]
keySplitWords = [ nltk.word_tokenize( str(sentence) ) for sentence in keySplit ]
full_df["studentSplit"] = studentSplitWords
full_df["keySplit"] = keySplitWords

In [14]:
#POS Tagging

full_df["keyPostag"] = full_df["keySplit"].apply(nltk.pos_tag)
full_df["studentPostag"] = full_df["studentSplit"].apply(nltk.pos_tag)

full_df[['keyPostag','studentPostag']]

,keyPostag,studentPostag
0,"[(simulate, VB), (behaviour, NN), (portions, N...","[(high, JJ), (risk, NN), (problems, NNS), (add..."
1,"[(simulate, VB), (behaviour, NN), (portions, N...","[(simulate, JJ), (portions, NNS), (desired, VB..."
2,"[(simulate, VB), (behaviour, NN), (portions, N...","[(prototype, NN), (program, NN), (simulates, V..."
3,"[(simulate, VB), (behaviour, NN), (portions, N...","[(defined, VBN), (specification, NN), (phase, ..."
4,"[(simulate, VB), (behaviour, NN), (portions, N...","[(used, VBN), (let, NN), (users, NNS), (first,..."
...,...,...
408,"[(store, NN), (class, NN), (interface, NN), (i...","[(allow, JJ), (compilers, NNS), (recognize, VB..."
409,"[(store, NN), (class, NN), (interface, NN), (i...","[(promote, NN), (function, NN), (reusuablitly,..."
410,"[(store, NN), (class, NN), (interface, NN), (i...","[(class, NN), (definitions, NNS), (placed, VBD)]"
411,"[(store, NN), (class, NN), (interface, NN), (i...","[(header, NN), (file, NN), (usually, RB), (con..."


Lemmatizing

In [15]:
#Lemmatizing

lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
def lemmatize_words(text):
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

full_df["studentLemmatized"] = full_df["studentStopword"].apply(lambda text: lemmatize_words(text))
full_df["keyLemmatized"] = full_df["keyStopword"].apply(lambda text: lemmatize_words(text))
full_df[['studentStopword','studentLemmatized']]

,studentStopword,studentLemmatized
0,high risk problems address prototype program m...,high risk problem address prototype program ma...
1,simulate portions desired final product quick ...,simulate portion desire final product quick ea...
2,prototype program simulates behavior portions ...,prototype program simulate behavior portion de...
3,defined specification phase prototype stimulat...,define specification phase prototype stimulate...
4,used let users first idea completed program al...,use let user first idea complete program allow...
...,...,...
408,allow compilers recognize classes used elsewhere,allow compiler recognize class use elsewhere
409,promote function reusuablitly,promote function reusuablitly
410,class definitions placed,class definition place
411,header file usually contains class andor funct...,header file usually contain class andor functi...


**TF-IDF**

TF

In [16]:
#TF for Lemmatized

full_df['keyLemmatized'] = full_df['keyLemmatized'].astype(str)
full_df['studentLemmatized'] = full_df['studentLemmatized'].astype(str)

for i in range(len(full_df)):
  doc1=full_df['keyLemmatized'][i]
  doc2=full_df['studentLemmatized'][i]
  docs=(doc1,doc2)

  countvectorizer = CountVectorizer()
  terms = countvectorizer.fit_transform(docs)

  columns = countvectorizer.get_feature_names_out()
  tf = pd.DataFrame(terms.toarray(),index = ['keyLemmatized','studentLemmatized'], columns=columns)

  print(tf)

                   address  also  behaviour  company  desire  feasible  high  \
keyLemmatized            0     0          1        0       1         0     0   
studentLemmatized        1     1          0        1       0         1     1   

                   make  may  portion  ...  product  program  programme  \
keyLemmatized         0    0        1  ...        1        0          0   
studentLemmatized     1    1        0  ...        0        2          1   

                   prototype  risk  show  simulate  software  sure  use  
keyLemmatized              0     0     0         1         1     0    0  
studentLemmatized          2     1     1         0         1     1    1  

[2 rows x 22 columns]
                   behaviour  desire  easy  final  help  job  may  portion  \
keyLemmatized              1       1     0      0     0    0    0        1   
studentLemmatized          0       1     1      2     1    1    1        1   

                   problem  product  program  project

In [17]:
#TF for Stem

full_df['keyStem'] = full_df['keyStem'].astype(str)
full_df['studentStem'] = full_df['studentStem'].astype(str)

for i in range(len(full_df)):
  doc12=full_df['keyStem'][i]
  doc22=full_df['studentStem'][i]
  docs2=(doc12,doc22)
  countvectorizer2 = CountVectorizer()
  terms2 = countvectorizer2.fit_transform(docs2)

  columns2 = countvectorizer2.get_feature_names_out()
  tf2 = pd.DataFrame(terms2.toarray(),index = ['keyStem','studentStem'], columns=columns2)

  print(tf2)

             address  also  behaviour  compani  desir  feasibl  high  make  \
keyStem            0     0          1        0      1        0     0     0   
studentStem        1     1          0        1      0        1     1     1   

             may  portion  ...  product  program  programm  prototyp  risk  \
keyStem        0        1  ...        1        0         0         0     0   
studentStem    1        0  ...        0        2         1         2     1   

             show  simul  softwar  sure  use  
keyStem         0      1        1     0    0  
studentStem     1      0        1     1    1  

[2 rows x 22 columns]
             behaviour  desir  easi  final  help  job  may  portion  problem  \
keyStem              1      1     0      0     0    0    0        1        0   
studentStem          0      1     1      2     1    1    1        1        1   

             product  program  project  quick  see  simul  small  softwar  \
keyStem            1        0        0      0   

IDF

In [18]:
# IDF for Lemmatized

for i in range(len(full_df)):
    doc1=full_df['keyLemmatized'][i]
    doc2=full_df['studentLemmatized'][i]
    docs=(doc1,doc2)

    countvectorizer = CountVectorizer()
    terms = countvectorizer.fit_transform(docs)

    tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True) 
    tfidf_transformer.fit(terms)

    df_idf = pd.DataFrame(tfidf_transformer.idf_, index=countvectorizer.get_feature_names_out(),columns=["idf_weights"]) 

    print(df_idf)


           idf_weights
address       1.405465
also          1.405465
behaviour     1.405465
company       1.405465
desire        1.405465
feasible      1.405465
high          1.405465
make          1.405465
may           1.405465
portion       1.405465
possibly      1.405465
problem       1.405465
product       1.405465
program       1.405465
programme     1.405465
prototype     1.405465
risk          1.405465
show          1.405465
simulate      1.405465
software      1.000000
sure          1.405465
use           1.405465
           idf_weights
behaviour     1.405465
desire        1.000000
easy          1.405465
final         1.405465
help          1.405465
job           1.405465
may           1.405465
portion       1.000000
problem       1.405465
product       1.000000
program       1.405465
project       1.405465
quick         1.405465
see           1.405465
simulate      1.000000
small         1.405465
software      1.405465
solve         1.405465
specific      1.405465
way        

In [19]:
# IDF for Stem

for i in range(len(full_df)):
  doc12=full_df['keyStem'][i]
  doc22=full_df['studentStem'][i]
  docs2=(doc12,doc22)
  countvectorizer = CountVectorizer()
  terms2 = countvectorizer.fit_transform(docs2)

  tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True) 
  tfidf_transformer.fit(terms2)

  df_idf2 = pd.DataFrame(tfidf_transformer.idf_, index=countvectorizer.get_feature_names_out(),columns=["idf_weights"]) 

  print(df_idf2)

           idf_weights
address       1.405465
also          1.405465
behaviour     1.405465
compani       1.405465
desir         1.405465
feasibl       1.405465
high          1.405465
make          1.405465
may           1.405465
portion       1.405465
possibl       1.405465
problem       1.405465
product       1.405465
program       1.405465
programm      1.405465
prototyp      1.405465
risk          1.405465
show          1.405465
simul         1.405465
softwar       1.000000
sure          1.405465
use           1.405465
           idf_weights
behaviour     1.405465
desir         1.000000
easi          1.405465
final         1.405465
help          1.405465
job           1.405465
may           1.405465
portion       1.000000
problem       1.405465
product       1.000000
program       1.405465
project       1.405465
quick         1.405465
see           1.405465
simul         1.000000
small         1.405465
softwar       1.405465
solv          1.405465
specif        1.405465
way        

TF-IDF

In [20]:
# TF-IDF for Lemmatized

for i in range(len(full_df)):
    doc1=full_df['keyLemmatized'][i]
    doc2=full_df['studentLemmatized'][i]
    docs=(doc1,doc2)
    countvectorizer = CountVectorizer()
    terms = countvectorizer.fit_transform(docs)
    X = tfidf_transformer.fit_transform(terms)
    tf_idf = pd.DataFrame(X.toarray() ,columns=countvectorizer.get_feature_names_out())
    print(tf_idf)

    address      also  behaviour   company   desire  feasible      high  \
0  0.000000  0.000000    0.42616  0.000000  0.42616  0.000000  0.000000   
1  0.210789  0.210789    0.00000  0.210789  0.00000  0.210789  0.210789   

       make       may  portion  ...  product   program  programme  prototype  \
0  0.000000  0.000000  0.42616  ...  0.42616  0.000000   0.000000   0.000000   
1  0.210789  0.210789  0.00000  ...  0.00000  0.421579   0.210789   0.421579   

       risk      show  simulate  software      sure       use  
0  0.000000  0.000000   0.42616  0.303216  0.000000  0.000000  
1  0.210789  0.210789   0.00000  0.149978  0.210789  0.210789  

[2 rows x 22 columns]
   behaviour    desire      easy    final      help       job       may  \
0   0.498446  0.354649  0.000000  0.00000  0.000000  0.000000  0.000000   
1   0.000000  0.163124  0.229265  0.45853  0.229265  0.229265  0.229265   

    portion   problem   product   program   project     quick       see  \
0  0.354649  0.00

In [21]:
# TF-IDF for Stem

for i in range(len(full_df)):
    doc12=full_df['keyStem'][i]
    doc22=full_df['studentStem'][i]
    docs2=(doc12,doc22)
    countvectorizer = CountVectorizer()
    terms2 = countvectorizer.fit_transform(docs2)

    X2 = tfidf_transformer.fit_transform(terms2)
    tf_idf2 = pd.DataFrame(X2.toarray() ,columns=countvectorizer.get_feature_names_out())
    print(tf_idf2)

    address      also  behaviour   compani    desir   feasibl      high  \
0  0.000000  0.000000    0.42616  0.000000  0.42616  0.000000  0.000000   
1  0.210789  0.210789    0.00000  0.210789  0.00000  0.210789  0.210789   

       make       may  portion  ...  product   program  programm  prototyp  \
0  0.000000  0.000000  0.42616  ...  0.42616  0.000000  0.000000  0.000000   
1  0.210789  0.210789  0.00000  ...  0.00000  0.421579  0.210789  0.421579   

       risk      show    simul   softwar      sure       use  
0  0.000000  0.000000  0.42616  0.303216  0.000000  0.000000  
1  0.210789  0.210789  0.00000  0.149978  0.210789  0.210789  

[2 rows x 22 columns]
   behaviour     desir      easi    final      help       job       may  \
0   0.498446  0.354649  0.000000  0.00000  0.000000  0.000000  0.000000   
1   0.000000  0.163124  0.229265  0.45853  0.229265  0.229265  0.229265   

    portion   problem   product   program   project     quick       see  \
0  0.354649  0.000000  0.3

**Cosine Similarity**

In [22]:
# Cosine Similarity for Lemmatized

tfidf_vectorizer = TfidfVectorizer()
similarity=[]

full_df['keyLemmatized'] = full_df['keyLemmatized'].astype(str)
full_df['studentLemmatized'] = full_df['studentLemmatized'].astype(str)

for i in range(len(full_df)):
  doc1=full_df['keyLemmatized'][i]
  doc2=full_df['studentLemmatized'][i]
  docs=(doc1,doc2)
  tfidf_matrix = tfidf_vectorizer.fit_transform(docs)
  tfidf_tokens = tfidf_vectorizer.get_feature_names_out()
  df_tfidfvect = pd.DataFrame(data = tfidf_matrix.toarray(),index = ['keyLemmatized','studentLemmatized'],columns = tfidf_tokens)
  print(df_tfidfvect)

  cosine_sim = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])
  print("Cosine Similarity =", cosine_sim[0][0])
  print("\n")
  similarity.append(cosine_sim[0][0])

full_df['cosineSimilarityLemma']=similarity

# cosinelemma = full_df['cosineSimilarityLemma']
# cosinelemma.to_csv('cosinelemma.csv')

                    address      also  behaviour   company   desire  feasible  \
keyLemmatized      0.000000  0.000000    0.42616  0.000000  0.42616  0.000000   
studentLemmatized  0.210789  0.210789    0.00000  0.210789  0.00000  0.210789   

                       high      make       may  portion  ...  product  \
keyLemmatized      0.000000  0.000000  0.000000  0.42616  ...  0.42616   
studentLemmatized  0.210789  0.210789  0.210789  0.00000  ...  0.00000   

                    program  programme  prototype      risk      show  \
keyLemmatized      0.000000   0.000000   0.000000  0.000000  0.000000   
studentLemmatized  0.421579   0.210789   0.421579  0.210789  0.210789   

                   simulate  software      sure       use  
keyLemmatized       0.42616  0.303216  0.000000  0.000000  
studentLemmatized   0.00000  0.149978  0.210789  0.210789  

[2 rows x 22 columns]
Cosine Similarity = 0.04547582924081302


                   behaviour    desire      easy    final      help 

In [23]:
# Cosine Similarity for Stem

tfidf_vectorizer2 = TfidfVectorizer()
similarity2=[]

full_df['keyStem'] = full_df['keyStem'].astype(str)
full_df['studentStem'] = full_df['studentStem'].astype(str)

for i in range(len(full_df)):
  doc12=full_df['keyStem'][i]
  doc22=full_df['studentStem'][i]
  docs2=(doc12,doc22)
  tfidf_matrix2 = tfidf_vectorizer2.fit_transform(docs2)
  tfidf_tokens2 = tfidf_vectorizer2.get_feature_names_out()
  df_tfidfvect2 = pd.DataFrame(data = tfidf_matrix2.toarray(),index = ['keyStem','studentStem'],columns = tfidf_tokens2)
  print(df_tfidfvect2)

  cosine_sim2 = cosine_similarity(tfidf_matrix2[0], tfidf_matrix2[1])
  print("Cosine Similarity =", cosine_sim2[0][0])
  print("\n")
  similarity2.append(cosine_sim2[0][0])

full_df['cosineSimilarityStem']=similarity2

              address      also  behaviour   compani    desir   feasibl  \
keyStem      0.000000  0.000000    0.42616  0.000000  0.42616  0.000000   
studentStem  0.210789  0.210789    0.00000  0.210789  0.00000  0.210789   

                 high      make       may  portion  ...  product   program  \
keyStem      0.000000  0.000000  0.000000  0.42616  ...  0.42616  0.000000   
studentStem  0.210789  0.210789  0.210789  0.00000  ...  0.00000  0.421579   

             programm  prototyp      risk      show    simul   softwar  \
keyStem      0.000000  0.000000  0.000000  0.000000  0.42616  0.303216   
studentStem  0.210789  0.421579  0.210789  0.210789  0.00000  0.149978   

                 sure       use  
keyStem      0.000000  0.000000  
studentStem  0.210789  0.210789  

[2 rows x 22 columns]
Cosine Similarity = 0.04547582924081302


             behaviour     desir      easi    final      help       job  \
keyStem       0.498446  0.354649  0.000000  0.00000  0.000000  0.000000   

**Scoring**

In [24]:
# Scoring for Lemmatized
scoring = full_df['cosineSimilarityLemma']

scoring = scoring.mul(5)

# full_df = full_df.drop('scoreModel', axis=1)
full_df['scoreModelLemma'] = scoring

full_df[['scoreModelLemma']]

,scoreModelLemma
0,0.227379
1,1.157034
2,2.305855
3,1.100628
4,0.217903
...,...
408,0.332796
409,0.493281
410,0.493281
411,0.964007


In [25]:
# Scoring for Stemming
scoring2 = full_df['cosineSimilarityStem']

scoring2 = scoring2.mul(5)

full_df['scoreModelStem'] = scoring2

full_df[['scoreModelStem']]

,scoreModelStem
0,0.227379
1,1.157034
2,2.305855
3,1.100628
4,0.217903
...,...
408,0.332796
409,0.493281
410,0.493281
411,0.964007


Uji Analisis (Abaikan)

index 11, index 31

In [26]:
full_df[['keyAnswer','studentAnswer']].iloc[[31]]

,keyAnswer,studentAnswer
31,The testing stage can influence both the codin...,"Verification, coding, refining the solution an..."


In [27]:
full_df[['keyStopword','studentStopword']].iloc[[31]]

,keyStopword,studentStopword
31,testing stage influence coming stage phase 5 s...,purification coming refining solution maintena...


In [28]:
test = full_df[['keyPostag','studentPostag']].iloc[[31]]
test.to_csv('cosinelemma.csv')

In [29]:
full_df[['keyStem','studentStem']].iloc[[31]]

,keyStem,studentStem
31,test stage influenc come stage phase 5 solut r...,purif come refin solut mainten influenc test s...


In [30]:
full_df[['keyLemmatized','studentLemmatized']].iloc[[31]]

,keyLemmatized,studentLemmatized
31,test stage influence come stage phase 5 soluti...,purification come refine solution maintenance ...


In [31]:
full_df.iloc[30:40]

,scoreTeacher,studentId,question,keyAnswer,studentAnswer,keySpell,studentSpell,keyLower,studentLower,keyPunct,...,studentSplit,keySplit,keyPostag,studentPostag,studentLemmatized,keyLemmatized,cosineSimilarityLemma,cosineSimilarityStem,scoreModelLemma,scoreModelStem
30,4.0,[5],What stages in the software life cycle are inf...,The testing stage can influence both the codin...,The implementation phase and the maintenance p...,The testing stage can influence both the comin...,The implementation phase and the maintenance p...,the testing stage can influence both the comin...,the implementation phase and the maintenance p...,the testing stage can influence both the comin...,...,"[implementation, phase, maintenance, phase, ef...","[testing, stage, influence, coming, stage, pha...","[(testing, VBG), (stage, NN), (influence, NN),...","[(implementation, NN), (phase, NN), (maintenan...",implementation phase maintenance phase effect,test stage influence come stage phase 5 soluti...,0.225660,0.225660,1.128299,1.128299
31,4.5,[8],What stages in the software life cycle are inf...,The testing stage can influence both the codin...,"Verification, coding, refining the solution an...",The testing stage can influence both the comin...,"Purification, coming, refining the solution an...",the testing stage can influence both the comin...,"purification, coming, refining the solution an...",the testing stage can influence both the comin...,...,"[purification, coming, refining, solution, mai...","[testing, stage, influence, coming, stage, pha...","[(testing, VBG), (stage, NN), (influence, NN),...","[(purification, NN), (coming, VBG), (refining,...",purification come refine solution maintenance ...,test stage influence come stage phase 5 soluti...,0.356116,0.541912,1.780579,2.709560
32,3.0,[3],What stages in the software life cycle are inf...,The testing stage can influence both the codin...,In RUP the stages in the software life cycle a...,The testing stage can influence both the comin...,In RUP the stages in the software life cycle a...,the testing stage can influence both the comin...,in rup the stages in the software life cycle a...,the testing stage can influence both the comin...,...,"[rup, stages, software, life, cycle, influence...","[testing, stage, influence, coming, stage, pha...","[(testing, VBG), (stage, NN), (influence, NN),...","[(rup, NN), (stages, NNS), (software, NN), (li...",rup stage software life cycle influence test s...,test stage influence come stage phase 5 soluti...,0.278042,0.310191,1.390211,1.550957
33,3.0,[4],What stages in the software life cycle are inf...,The testing stage can influence both the codin...,"Refining the solution, Production and Maintena...",The testing stage can influence both the comin...,"Defining the solution, Production and Maintena...",the testing stage can influence both the comin...,"defining the solution, production and maintena...",the testing stage can influence both the comin...,...,"[defining, solution, production, maintenance, ...","[testing, stage, influence, coming, stage, pha...","[(testing, VBG), (stage, NN), (influence, NN),...","[(defining, VBG), (solution, NN), (production,...",define solution production maintenance influen...,test stage influence come stage phase 5 soluti...,0.303924,0.303924,1.519621,1.519621
34,2.0,[24],What stages in the software life cycle are inf...,The testing stage can influence both the codin...,"Elaboration, Construction, and Transition are ...",The testing stage can influence both the comin...,"Elaboration, Construction, and Transition are ...",the testing stage can influence both the comin...,"elaboration, construction, and transition are ...",the testing stage can influence both the comin...,...,"[elaboration, construction, transition, affect...","[testing, stage, influence, coming, stage, pha...","[(testing, VBG), (stage, NN), (influence, NN),...","[(elaboration, NN), (construction, NN), (trans...",elaboration construction transition affect test,test stage i

**Mean Absolute Error**

In [32]:
# Mean Absolute Error for Lemmatized

sum = 0
actual = full_df['scoreTeacher']
calculated = full_df['scoreModelLemma']
n = len(full_df)
  
# for loop for iteration
for i in range(n):
    sum += abs(actual[i] - calculated[i])
  
maeLemma = sum/n
maeLemma

2.6420121125293736

In [33]:
# Mean Absolute Error for Stem

sum2 = 0
actual2 = full_df['scoreTeacher']
calculated2 = full_df['scoreModelStem']
n2 = len(full_df)
  
# for loop for iteration
for i in range(n2):
    sum2 += abs(actual2[i] - calculated2[i])
  
maeStem = sum2/n2
maeStem

2.5938198213004067